In [ ]:
mp.set_start_method("spawn", force=True)  # safest cross-platform

rpc = make_worker_rpc()

p = mp.Process(target=worker_main, args=(rpc,), daemon=True)
p.start()

# Main-side handler for worker calls
config = {"threshold": 10}

def handler(kind: str, payload: Any) -> Any:
    if kind == "get_config":
        return {"threshold": config["threshold"]}

    if kind == "should_process":
        x = int(payload["x"])
        threshold = int(payload["threshold"])
        if x < threshold:
            return {"ok": False, "reason": f"{x} < {threshold}"}
        return {"ok": True}

    if kind == "report_result":
        # You can do anything here: write DB, update UI, etc.
        return f"stored({payload['x']}->{payload['result']})"

    raise ValueError(f"Unknown RPC kind: {kind!r}")

# Serve requests while also printing signals
# (Here we interleave: drain signals frequently; handle RPC in serve loop.)
# Simple approach: run serve() in a thread so main can do other stuff.
import threading

server_thread = threading.Thread(
    target=rpc.serve,
    args=(handler,),
    kwargs={"stop_when": lambda: (p.exitcode is not None)},
    daemon=True,
)
server_thread.start()

# Meanwhile, consume signals until worker exits
while p.is_alive() or (rpc.recv_signal(timeout=0.05) is not None):
    msg = rpc.recv_signal(timeout=0.05)
    if msg:
        print("MAIN signal:", msg)

p.join()
server_thread.join(timeout=1)
print("worker exitcode:", p.exitcode)


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/lukastk/.local/share/uv/python/cpython-3.11.11-macos-aarch64-none/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/lukastk/.local/share/uv/python/cpython-3.11.11-macos-aarch64-none/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'worker_fn' on <module '__main__' (built-in)>


worker exitcode: 1
